In [1]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

# Load your dataset
df = pd.read_csv('dataset_space_API_past_launches_raw.csv')  # Replace with your dataset path
df = df[df['BoosterVersion'] == 'Falcon 9']
df['PayloadMass'].fillna(df['PayloadMass'].mean(), inplace=True)
# Function to classify landing outcome as successful (1) or unsuccessful (0)
def classify_landing_outcome(outcome):
    if 'True' in outcome:
        return 1
    else:
        return 0

# Create a 'LandingOutcome' column
df['LandingOutcome'] = df['Outcome'].apply(classify_landing_outcome)

# Initialize your Dash app
app = dash.Dash(__name__)

# App layout
app.layout = html.Div([
    # Dropdown for selecting a launch site
    dcc.Dropdown(
        id='launch-site-dropdown',
        options=[{'label': i, 'value': i} for i in df['LaunchSite'].unique()],
        value='All Sites'
    ),
    # Pie chart for launch site success rate
    dcc.Graph(id='success-rate-pie-chart'),

    # Range slider for payload mass
    dcc.RangeSlider(
        id='payload-range-slider',
        min=df['PayloadMass'].min(),
        max=df['PayloadMass'].max(),
        value=[df['PayloadMass'].min(), df['PayloadMass'].max()],
        marks={str(i): str(i) for i in range(int(df['PayloadMass'].min()), int(df['PayloadMass'].max()), 1000)}
    ),
    # Graphs for payload mass vs. landing outcome, payload mass for all sites
    dcc.Graph(id='payload-mass-landing-outcome-graph'),
    dcc.Graph(id='payload-mass-all-sites-graph')
])

# Callback for updating the pie chart
@app.callback(
    Output('success-rate-pie-chart', 'figure'),
    Input('launch-site-dropdown', 'value'))
def update_pie_chart(selected_site):
    if selected_site == 'All Sites':
        filtered_df = df
    else:
        filtered_df = df[df['LaunchSite'] == selected_site]
    fig = px.pie(filtered_df, names='LandingOutcome', title='Landing Success Rate')
    return fig

# Callback for updating the payload mass graphs
@app.callback(
    [Output('payload-mass-landing-outcome-graph', 'figure'),
     Output('payload-mass-all-sites-graph', 'figure')],
    Input('payload-range-slider', 'value'))
def update_payload_graphs(payload_range):
    filtered_df = df[(df['PayloadMass'] >= payload_range[0]) & (df['PayloadMass'] <= payload_range[1])]
    fig1 = px.scatter(filtered_df, x='PayloadMass', y='LandingOutcome', color='LaunchSite', title='Payload Mass vs. Landing Outcome')
    fig2 = px.scatter(filtered_df, x='PayloadMass', y='LaunchSite', color='LandingOutcome', title='Payload Mass vs. Launch Site')
    return fig1, fig2

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\59704\AppData\Local\Temp/ipykernel_1676508/3594559953.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\59704\AppData\Local\Temp/ipykernel_1676508/3594559953.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
